# Extracting Sentinel-2 indices time series from Google Earth Engine

# Step 1. Import required modules

In [ ]:
import geemap
import ee
import numpy
import eemont
import csv
import os
import io
import requests
import pandas
from osgeo import ogr

print('\nGoogle Earth Engine initialize..')
ee.Initialize()
print('Done.\n')

Map = geemap.Map(center=[41.8,12.5], zoom=2)
url = 'http://mt0.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}'
Map.add_tile_layer(url, name='Google Map Hybrid', attribution='Google')
Map

# Step 2. Input 4 mandatory parameters defined by the user: 

### Path of fields of interest file

POLYGON_PATH: ["DRAW" | shapefile path | geoson file path]

Note: Choose "DRAW" to draw the polygon on the MAP ABOVE or write the path of a shapefile (.shp) or a .geojson file. If you chose "DRAW", draw the polygon/polygons on the map before moving forward.

### Date range (initial and final dates of the analysis)

date1: ['YYYY-MM-DD'] & date2: ['YYYY-MM-DD']

### Path of output file with Vegetation Indices statistics

OUTPUT_CSV: [path of output .csv files]

### Anomaly type

anomaly: ['true' | 'false' | 'topredict']

Note: 
Choose anomaly 'true' or 'false' to create the datasets required for the model training ['true' for fields related to the presence of an anomaly (i.e. cause by pest) or 'false' in case are not related to any anomaly], 
Or choose anomaly 'topredict' in case the fields need to be analyzed based on a already trained model.

In [ ]:
# INPUTS to be defined by the user

# Define Fields of Interest File path
# *** write "DRAW" if you want to draw the polygon on the MAP ABOVE
# *** or write the path of a .SHP or .GEOJSON file
POLYGON_PATH= r'C:\Users\Utente\Documents\PSA\test\roi\wheat_tests.shp'

# SELECT THE INPUT DATES
date1='2022-10-01' # year-month-day FORMAT 'YYYY-MM-DD'
date2='2022-12-01' # year-month-day FORMAT 'YYYY-MM-DD'

# SELECT ANOMALY PRESENCE
anomaly='true' # 'true' or 'false' or 'topredict'

# Define CSV output directory (path with '/' at the end)
OUTDIR_CSV='C:/Users/Utente/Documents/PSA/test/CSV/'

#------------------------------------------------------------------------------------------#
print('SELECTED PARAMETERS\n')
print('    INPUT POLYGON_PATH:  {}'.format(POLYGON_PATH) )
# if you chose "DRAW", draw the polygon/polygons on the previous map before moving forward
if POLYGON_PATH=="DRAW":
    print('If you chose "DRAW", draw the polygon/polygons on the map before moving forward')
    roiname='roi'
else:
    roiname=os.path.splitext(os.path.basename(POLYGON_PATH))[0]
print('    ROI name : {}\n'.format(roiname))
print('    INPUT DATE RANGE: {} - {}\n'.format(date1,date2) )
print('    ANOMALY: {}\n'.format(anomaly) )
print('Step finished.\n')

# Step 3. Region of interest visualization 

In [ ]:
# CASE 1 : DRAW YOUR REGION OF INTEREST (POLYGON) ON THE MAP ABOVE
if POLYGON_PATH=="DRAW": # Before running this part you have to select and draw at least 1 polygon on the previous Map
    Map.draw_features
    roi = ee.FeatureCollection(Map.draw_features)
    feat_collection=ee.FeatureCollection(roi)
    roi_geojson=geemap.ee_to_geojson(roi)
    #print(roi_geojson)
    geemap.ee_to_geojson(roi, filename='Region_of_interest.geojson')
    roi_path='Region_of_interest.geojson'
    
# CASE 2: Using a SHAPEFILE AS REGION OF INTEREST
elif POLYGON_PATH.endswith('.shp'):
    path_shp =POLYGON_PATH
    roi=geemap.shp_to_ee(path_shp)
    feat_collection=ee.FeatureCollection(geemap.shp_to_ee(path_shp))
    roi_path=path_shp
    
# CASE 3: Using a GEOJSON FILE AS REGION OF INTEREST
elif POLYGON_PATH.endswith('.geojson'):
    geojson_path='Region_of_interest.geojson'
    roi=geemap.geojson_to_ee(geojson_path)
    feat_collection=ee.FeatureCollection(roi)
    roi_path=geojson_path

print('Done')
    
Map.clear()
Map.add_tile_layer(url, name='Google Map Hybrid', attribution='Google')
Map.addLayer(feat_collection)
Map.centerObject(feat_collection, 16)
Map    
    

# Step 4. Functions definition

In [ ]:
# Functions

def addIndices(img):
    N =  img.select('B8')
    R =  img.select('B4') 
    B =  img.select('B2') 
    RE1 =  img.select('B5') 
    RE2 =  img.select('B6') 
    G =  img.select('B3') 
    modNDRE = ee.Image().expression(
             "(N-RE1)/(N+RE1-(2**B))",
             {"N": N, "RE1": RE1, "B": B} ).rename("modNDRE")
    PVR = ee.Image().expression(
             "(G/R)",
             {"G": G, "R": R} ).rename("PVR")
    GCI = ee.Image().expression(
             "(N/G) - 1",
             {"G": G, "N": N} ).rename("GCI")
    LCI = ee.Image().expression(
             "(N-RE1) / (N+R)",
             {"R": R,"RE1": RE1, "N": N} ).rename("LCI")
    TCVI = ee.Image().expression(
             "-0.283*G - 0.660*R + 0.577*RE1 + 0.388*N",
             {"R": R,"RE1": RE1, "N": N, "G": G} ).rename("TCVI")
    SIPI1 = ee.Image().expression(
             " (N-B)/(N-R)",
             {"R": R,"B": B, "N": N} ).rename("SIPI1")
    SIPI2 = ee.Image().expression(
             " (N-B)/(N-R)",
             {"R": R,"B": B, "N": N} ).rename("SIPI2")   
    return img.addBands([modNDRE,PVR,GCI,LCI,TCVI,SIPI1,SIPI2])                    

def getlist_unique_fromdataframe(df,field_str):
    df_date=df[field_str]
    dlist=[]
    for i in range(0, len(df_date)):
        update=df_date[i]
        if update not in dlist:
            dlist.append(update)    
    return dlist

def getindexlist_fromdataframe_eqvalue(df,field_str,value):
    df_date=df[field_str]
    indexlist=[]
    for i in range(0, len(df_date)):
        update=df_date[i]
        if update == value:
            indexlist.append(i)    
    return indexlist

def getindexlist_fromdataframe_includestring(df,field_str,value):
    df_date=df[field_str]
    indexlist=[]
    for i in range(0, len(df_date)):
        update=df_date[i]
        if value in update:
            indexlist.append(i)    
    return indexlist

def getindexlist_fromdataframe_filter_date_ID(df,date_,id_):
    idxlist=[]
    dateindex=getindexlist_fromdataframe_eqvalue(df,'date',date_)
    idindex=getindexlist_fromdataframe_eqvalue(df,'id',id_) 
    idxlist= extract_common_list(dateindex,idindex)
    return idxlist

def getindexlist_fromdataframe_filter_date_ID_reducer(df,date_,id_, reducer_):
    idxlist=[]
    dateindex=getindexlist_fromdataframe_eqvalue(df,'date',date_)
    idindex=getindexlist_fromdataframe_eqvalue(df,'id',id_) 
    redindex=getindexlist_fromdataframe_eqvalue(df,'reducer',reducer_) 
    list_0=extract_common_list(dateindex, idindex)
    idxlist= extract_common_list(list_0,redindex)
    return idxlist

def getindexlist_fromdataframe_filter_dateday_ID_reducer(df,date_,id_,reducer_):
    idxlist=[]
    dateindex=getindexlist_fromdataframe_includestring(df,'date',date_)
    idindex=getindexlist_fromdataframe_eqvalue(df,'id',id_) 
    redindex=getindexlist_fromdataframe_eqvalue(df,'reducer',reducer_) 
    list_0=extract_common_list(dateindex, idindex)
    idxlist= extract_common_list(list_0,redindex)
    return idxlist

def extract_common_list(list1, list2):
    commonlist=[]
    for i in range(0, len(list1)):
        update=list1[i]
        if update in list2 :
            commonlist.append(update)    
    return commonlist

def correct_nan_values(mean_,min_,max_,std_):
    if numpy.isnan(mean_ ) or numpy.isnan(min_ )or numpy.isnan(max_ )or numpy.isnan(std_ ) :
        mean_=numpy.nan
        min_=numpy.nan
        max_=numpy.nan
        std_=numpy.nan
    return mean_,min_,max_,std_

def get_unique_date_by_day_list(datelist_):
    filtered_datelist=[]
    for i in range(0, len(datelist_)):
        update=datelist_[i][0:10]
        if update not in filtered_datelist :
            filtered_datelist.append(update) 
    return filtered_datelist

def get_unique_date_by_datetime_list(datelist_):
    filtered_datelist=[]
    for i in range(0, len(datelist_)):
        update=datelist_[i]
        if update not in filtered_datelist :
            filtered_datelist.append(update) 
    return filtered_datelist

# Step 5. GEE image collection (Sentinel-2) processing
### Extraction of Field Statitics (Mean, Minimum, Maximum, Standard Deviation)

In [ ]:
# GEE image collection (Sentinel-2) + Preprocessing + Statitics extraction

# ROI 
feat_collection=roi

# INDICES
# Supported by spectralIndices library in GEE eemont
spectralIndices_list= [ "NDVI","SR","GNDVI","NDREI","EVI","EVI2","CIG","CIRE","TriVI","MTCI","GARI","SIPI","MCARI","ARVI", "OSAVI"]
# Unsupported by spectralIndices library, builded by the function addIndices()
addIndices_list = ["modNDRE","PVR","GCI","LCI","TCVI","SIPI1","SIPI2"]
# FInal order of the CSV visualization (all indexes)
all_index_list= [ "SR","NDVI","GNDVI","NDREI","modNDRE","EVI","EVI2","PVR","GCI","CIRE","TriVI","MTCI","LCI","TCVI","GARI","SIPI1","SIPI2","MCARI","ARVI", "OSAVI"]

# Google Earth Engine: get IMAGE COLLECTION (SENTINEL-2) ----------------------
# A Sentinel-2 surface reflectance image.
print('Collecting Sentinel-2 images from GEE... ')
ds = 'COPERNICUS/S2_SR'         
S2 = (ee.ImageCollection(ds)
     #.closest(date).first().scale().filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 40)).maskClouds(prob = 75,buffer = 300,cdi = -0.5)
    .filterBounds(feat_collection)
    .scaleAndOffset()
    .filterDate(date1, date2)
    .maskClouds()
    .spectralIndices(spectralIndices_list)
    .map(addIndices) )
print('Done.\n')

# Get statistics --------------------------------------------------------------
print('Extracting zonal statistics time-series from GEE image-collection ...')
ts = S2.getTimeSeriesByRegions(reducer = [ee.Reducer.mean(),ee.Reducer.min(),ee.Reducer.max(),ee.Reducer.stdDev()], 
                              collection = feat_collection,
                              bands = all_index_list,
                              naValue = -9999,
                              scale = 10)
print('Step finished.\n')


# Step 6. Downloading of fields statistics

In [ ]:
# Downloading and Reading stats as pandas Dataframe.

dirpath=OUTDIR_CSV
if not os.path.isdir(dirpath):
    os.mkdir(dirpath)   
print('Downloading and Reading stats as pandas Dataframe...')
# gdf = geemap.ee_to_pandas(ts)
url = ts.getDownloadURL(
    filetype="csv",
    selectors=['id','reducer','date',"NDVI","SR","GNDVI","NDREI","modNDRE","EVI","EVI2","PVR","GCI","CIRE","TriVI","MTCI","LCI","TCVI","GARI","SIPI1","SIPI2","MCARI","ARVI","OSAVI"],#+all_index_list
    filename="ts")
print(url)
temp_path=os.path.join(OUTDIR_CSV,'csv_from_url.csv')
response = requests.get(url)
print('CSV written.\n')
with open(temp_path, 'wb') as fd:
    fd.write(response.content)  
print('Reference CSV path : {}'.format(temp_path))
print('Step finished.\n')


# Step 7. Export fields statistics
### output: fields_anomaly_[anomaly].csv 

In [ ]:
# Writing CSV file
# Manipulating the information inside the pandas dataframe to obtain the desired CSV visualization and format

gdf=pandas.read_csv(temp_path)
gdf[gdf == -9999] = numpy.nan
datelist=(getlist_unique_fromdataframe(gdf,'date'))
#print('Date list : {}'.format(datelist))

# Open the shapefile and get the first layer
datasource = ogr.Open(roi_path)
layer = datasource.GetLayerByIndex(0)
featurecount=layer.GetFeatureCount()
print("Number of features:{}".format(layer.GetFeatureCount()))
Layer=None
datasource=None
length=len(gdf['date'])/4/featurecount
idlist=getlist_unique_fromdataframe(gdf,'id')
number_of_rows_per_stats=gdf['reducer'].loc[lambda x: x=='min'].index[0]
img_number=int(len(getlist_unique_fromdataframe(gdf,'date')))
multiple_number_stats=int((len(gdf['date'])/4))
num_of_polygons=int(featurecount)
uniquedaylist=get_unique_date_by_day_list(getlist_unique_fromdataframe(gdf,'date'))
print('Date list : {}'.format(uniquedaylist))

#check ID values
if numpy.nan in gdf['id']:
    id_column=numpy.arange(0,int((num_of_polygons))).tolist()*(len(idlist))
    gdf['id']=id_column
    idlist=getlist_unique_fromdataframe(gdf,'id')

#check NaN values    
for z in gdf['id']:
    if numpy.isnan(z) :
        id_column=numpy.arange(0,int((num_of_polygons))).tolist()*(len(idlist))
        gdf['id']=id_column
        idlist=getlist_unique_fromdataframe(gdf,'id')
        break
        
print('Writing CSV file...')
all_index_list2 = [word.replace('NDREI','NDRE') for word in all_index_list]
all_index_list2 = [word.replace('CIRE','RECI') for word in all_index_list2]
all_index_list2 = [word.replace('TriVI','TVI') for word in all_index_list2]
all_index_list2 = [word.replace('GARI', 'GARVI') for word in all_index_list2]         
basenamecsv='fields_anomaly_' + anomaly
filepathcsv=os.path.join(OUTDIR_CSV,basenamecsv+'.csv')
if os.path.isfile(filepathcsv):
    os.remove(filepathcsv)   
# creation of Header
list_fields=['date']
for key in all_index_list2:
    list_fields=list_fields+list([key+'_MIN'])+list([key+'_MAX'])+list([key+'_MEAN'])+list([key+'_STD'])            
list_fields.append('anomaly')
with open(filepathcsv, 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(list_fields)

for k in idlist:
    #filepathcsv=os.path.join(OUTDIR_CSV,basenamecsv+str(k)+'.csv')        
    daylist=[]
    for date in datelist:
        day=date[0:10]
        if not day in daylist:
            daylist.append(day)
            list_csv=[]
            list_csv.append(date[0:10])#
            for key in all_index_list:
                    idx0=getindexlist_fromdataframe_filter_dateday_ID_reducer(gdf,date,k,'mean')[0]
                    meanv= gdf[key][idx0]
                    minv= gdf[key][idx0+multiple_number_stats]
                    maxv= gdf[key][idx0+multiple_number_stats*2]
                    stdv= gdf[key][idx0+multiple_number_stats*3]
                    meanv, minv, maxv, stdv=correct_nan_values(meanv,minv,maxv,stdv)
                    list_csv.append(minv)
                    list_csv.append(maxv)
                    list_csv.append(meanv)
                    list_csv.append(stdv)
            list_csv.append(anomaly)
            with open(filepathcsv, 'a', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(list_csv)
print('File written:  {}\n'.format(filepathcsv))
print('Step finished. Output .csv file ready. \n')